# Blatt 03 - Programmieraufgabe
Implementieren Sie eine Funktion, die die Cholesky Zerlegung implementiert

In [37]:
import numpy as np

def cholesky_fact(A):
    if A.shape[0] != A.shape[1] or A.shape[0] <= 1:
        raise Exception("expected square mat")

    n = A.shape[0]
    L = np.zeros((n, n))
    for s in range(n):
        sum_sqrs = 0 if s == 0 else np.sum([L[s, k]**2 for k in range(s)])
        
        L[s, s] = np.sqrt((A[s, s] - sum_sqrs))

        for i in range(s + 1, n):
            sum_prod = 0 if s == 0 else np.sum([L[s, k]*L[i, k] for k in range(s)])
            
            L[i, s] = (A[i, s] - sum_prod)/L[s, s]

    return L
        


In [38]:
A = np.array([[ 1,  2,  4],
              [ 2, 13, 23],
              [ 4, 23,77]])

L = cholesky_fact(A)
print(L @ L.T)

[[ 1.  2.  4.]
 [ 2. 13. 23.]
 [ 4. 23. 77.]]


## b) Formen Sie das Gleichungssystem von Blatt Zwei so um das Sie es mit der Cholesky Zerlegung und Vorwaerts- und Rueckwaertseinsetzen Loesen koennen und loesen Sie es.
Die Matrix aus dem 2. Blatt ist nicht Symmetrisch. Aus der linearen Algebra weiss man aber, dass man eine symmetrische Matrix draus gewinnen kann, in dem man die Matrix mit deren Transponierten multipliziert

In [65]:
B = np.array([[ 1,  3, -2],
              [-2, -5,  2],
              [ 3,  7, -7]])

d = np.array([2, -3, 9])

B_symm  = B.T @ B
print("B_symm:") 
print(B_symm)

B_symm:
[[ 14  34 -27]
 [ 34  83 -65]
 [-27 -65  57]]


Aus Lemma 3.26 kennen wir ein Kriterium, mit dem wir `B_symm` auf Positiv-Deifint-heit pruefen koennen.

In [55]:
def is_strict_diagdom(A):
    rows = A.shape[0]
    cols = A.shape[1]
    for r in range(rows):
        abs_sum = np.sum(np.abs(A[r, :])) - 2 * np.abs(A[r, r])
        if abs_sum > 0:
            return False

    return True

def is_diag_pos(A):
    rows = A.shape[0]
    for r in range(rows):
        if A[r, r] <= 0:
            return False
    return True


In [61]:
# Nun pruefen wir B
print("B_symm ist strikt diagdom und die Diagonal ist > 0?", is_diag_pos(B_symm) and is_strict_diagdom(B_symm))

B_symm ist strikt diagdom und die Diagonal ist > 0? False


Leider funktioniert das nicht. Wir pruefen die Eigenwerte.

In [90]:
eigvals_B_symm, eigvecs_B_symm = np.linalg.eig(B_symm)
print("B_symm Eigenwerte alle positiv?", eigvals_B_symm > 0) # Alle positiv. Wir machen weiter

f = B.T @ d # die rechte Seite muss man auch mit B^t multiplizieren

L = cholesky_fact(B_symm)

# Sicherstellen, dass es funktioniert
print("\nL L^ T = B_symm?")
print(L @ L.T == B_symm)

# Haben wir eine eigene Implementation aus Blatt 02? Ja. Steht da, dass man unbedingt die benutzen muss? Nein
from scipy.linalg import solve_triangular
z = solve_triangular(L, f, lower=True)
x = solve_triangular(L.T, z, lower=False)


print("Bx vs d", B @ x,"=", d)

B_symm Eigenwerte alle positiv? [ True  True  True]

L L^ T = B_symm?
[[ True  True  True]
 [ True  True  True]
 [ True  True  True]]
Bx vs d [ 2. -3.  9.] = [ 2 -3  9]
